In [5]:
import os
from transformers.file_utils import cached_path, hf_bucket_url
from importlib.machinery import SourceFileLoader
from transformers import EncoderDecoderModel
import argparse

cache_dir = './cache'
model_name = 'nguyenvulebinh/envibert'

def download_tokenizer_files():
    resources = ['envibert_tokenizer.py', 'dict.txt', 'sentencepiece.bpe.model']
    for item in resources:
        if not os.path.exists(os.path.join(cache_dir, item)):
            tmp_file = hf_bucket_url(model_name, filename=item)
            tmp_file = cached_path(tmp_file, cache_dir=cache_dir)
            os.rename(tmp_file, os.path.join(cache_dir, item))

def init_tokenizer():
    download_tokenizer_files()
    tokenizer = SourceFileLoader("envibert.tokenizer",
                                 os.path.join(cache_dir,
                                              'envibert_tokenizer.py')).load_module().RobertaTokenizer(cache_dir)
    return tokenizer

def init_model():
    if not os.path.exists(cache_dir):
        os.makedirs(cache_dir)
    download_tokenizer_files()
    tokenizer = SourceFileLoader("envibert.tokenizer",
                                 os.path.join(cache_dir,
                                              'envibert_tokenizer.py')).load_module().RobertaTokenizer(cache_dir)
    # set encoder decoder tying to True
    roberta_shared = EncoderDecoderModel.from_encoder_decoder_pretrained(model_name,
                                                                         model_name,
                                                                         tie_encoder_decoder=False)

    # set special tokens
    roberta_shared.config.decoder_start_token_id = tokenizer.bos_token_id
    roberta_shared.config.eos_token_id = tokenizer.eos_token_id
    roberta_shared.config.pad_token_id = tokenizer.pad_token_id

    # sensible parameters for beam search
    # set decoding params
    roberta_shared.config.max_length = 100
    roberta_shared.config.early_stopping = True
    roberta_shared.config.no_repeat_ngram_size = 3
    roberta_shared.config.length_penalty = 2.0
    roberta_shared.config.num_beams = 1
    roberta_shared.config.vocab_size = roberta_shared.config.encoder.vocab_size

    return roberta_shared, tokenizer

trained_model, tokenizer = init_model()
trained_model = trained_model.from_pretrained("datnth1709/VietAI-NLP-ITN")

Downloading:   0%|          | 0.00/702 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/270M [00:00<?, ?B/s]

Some weights of the model checkpoint at nguyenvulebinh/envibert were not used when initializing RobertaModel: ['lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at nguyenvulebinh/envibert and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weigh

Downloading:   0%|          | 0.00/594M [00:00<?, ?B/s]

In [12]:
SRC_MAX_LENGTH = 100
TGT_MAX_LENGTH = 100

file = "input.txt"
with open(file, mode='r', encoding='utf-8') as f:
    text = f.read()

print("Text input: ")
print(text)
print(100*"-")

input_ids = tokenizer(text, max_length=SRC_MAX_LENGTH, truncation=True, return_tensors='pt')["input_ids"]
# generate text without beam-search
outputs = trained_model.generate(
    input_ids, 
    max_length=SRC_MAX_LENGTH, 
    num_return_sequences=1, 
    early_stopping=True
)

print("Text predict: ")
for i, output in enumerate(outputs):
    output_pieces = tokenizer.convert_ids_to_tokens(output.numpy().tolist())
    output_text = tokenizer.sp_model.decode(output_pieces)
    print(output_text)

Text input: 
thời báo kinh tế sài gòn online bộ trưởng âm chín chín phẩy chín tám bộ giao thông vận tải hồ nghĩa dũng đã ký quyết định phê duyệt quy hoạch sân tám trăm tám mươi chín lít bay tại tỉnh an giang giai đoạn đến mười bốn giờ năm mươi ba và định hướng đến mồng một tháng bảy có tổng nhu cầu vốn dự kiến là tám chín ba sáu tám một ba bẩy bốn năm bốn tỉ đồng bình nguyên dự án sân bay tại xã cần đăng huyện châu thành sẽ được thực hiện thành một ngàn tám trăm ba mươi bẩy giai đoạn với số vốn cần cho giai đoạn đến bốn giờ không phút là âm tám tám chấm không năm mươi tỉ đồng và giai đoạn định hướng đến một nghìn sáu trăm là âm chín ba ngàn hai trăm năm bốn phẩy bốn nghìn chín mươi bẩy tỉ đồng theo quyết định phê duyệt quy hoạch số i gi gi rờ lờ chín không không đến ngày mười hai sân bay an giang sẽ có một đường hạ cất cánh dài hai mươi ba ngàn bẩy trăm mười bốn phẩy ba nghìn tám trăm sáu mươi chín mét và rộng cộng bốn hai sáu sáu sáu không năm bốn sáu một mét đảm bảo cho các hoạt động

In [17]:
file = "input.txt"
with open(file, mode='r', encoding='utf-8') as f:
    text = f.read()
n = 100
text = text.split(" ")
new = [text[i:i+n] for i in range(0, len(text), n)]
for s in new:
    s = ' '.join(s)
    print(s)

thời báo kinh tế sài gòn online bộ trưởng âm chín chín phẩy chín tám bộ giao thông vận tải hồ nghĩa dũng đã ký quyết định phê duyệt quy hoạch sân tám trăm tám mươi chín lít bay tại tỉnh an giang giai đoạn đến mười bốn giờ năm mươi ba và định hướng đến mồng một tháng bảy có tổng nhu cầu vốn dự kiến là tám chín ba sáu tám một ba bẩy bốn năm bốn tỉ đồng bình nguyên dự án sân bay tại xã cần đăng huyện châu thành sẽ được thực hiện thành một
ngàn tám trăm ba mươi bẩy giai đoạn với số vốn cần cho giai đoạn đến bốn giờ không phút là âm tám tám chấm không năm mươi tỉ đồng và giai đoạn định hướng đến một nghìn sáu trăm là âm chín ba ngàn hai trăm năm bốn phẩy bốn nghìn chín mươi bẩy tỉ đồng theo quyết định phê duyệt quy hoạch số i gi gi rờ lờ chín không không đến ngày mười hai sân bay an giang sẽ có một đường hạ cất cánh dài hai mươi ba ngàn bẩy trăm mười bốn phẩy ba nghìn
tám trăm sáu mươi chín mét và rộng cộng bốn hai sáu sáu sáu không năm bốn sáu một mét đảm bảo cho các hoạt động khai thác củ